In [79]:
import os

def conf_checker(model,li_concentration,structure):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)
        
    check = []
    for x in range(0,15):
        directory=os.path.join(dir,"conf_"+str(x))
        if (os.path.exists(directory) == True):
            check.append(x)
    if check == []:
        check = [0]
    else:
        check = check
        
    return check

In [80]:

# Function to check the out file. A 1 indicates that the senctence 
# reached required accuracy - stopping structural energy minimisation' is present in the out file.

import os
from conf_checker import conf_checker

def out_checker(model,li_concentration,structure,functional_list,U_value,conf,configuration):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)

    try:
        if (len(U_value)>1):
            values = U_value    
    except:
        values = ([U_value])

    if (configuration == True):
        configurations=conf_checker(model,li_concentration,structure)
    else:
        configurations=[conf]



    if functional_list == 'pbeU':
        h=len(values)+1
        w=len(configurations)+1
        output = [[0 for x in range(w)] for y in range(h)] 
        q = 1
        p = 1
        for x in configurations:
            for i in values:
                directory=os.path.join(dir,"conf_"+str(x), 'pbeU',str(i))
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[p][q]=1
                            output[p][0]=values[p-1]
                p = p + 1
            p = 1
            output[0][q]=q-1
            q = q + 1

    elif functional_list == 'scan':
        try:
            h=2
            w=len(configurations)+1
            output = [[0 for x in range(w)] for y in range(h)] 
            q = 1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'scan')
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[1][q]=1
                            output[0][q]=x
                q = q + 1
        except:
            output = 'No SCAN calculation.'

    elif functional_list == 'hse':
        try:
            h=2
            w=len(configurations)+1
            output = [[0 for x in range(w)] for y in range(h)] 
            q = 1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'hse')
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[1][q]=1
                            output[0][q]=x
                q = q + 1
        except:
            output = 'No HSE calculation.'

    elif functional_list == ("pbeU", "scan" , "hse"):
        #pbeU
        h=len(values)+3
        w=len(configurations)+1
        output = [[0 for x in range(w)] for y in range(h)] 
        q = 1
        p = 1
        for x in configurations:
            for i in values:
                directory=os.path.join(dir,"conf_"+str(x), 'pbeU',str(i))
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[p][q]=1
                            output[p][0]=values[p-1]
                p = p + 1
            p = 1
            output[0][q]=q-1
            q = q + 1

        #scan
        try: 
            q = 1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'scan')
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[len(values)+1][q]=1
                q = q + 1

        except:
            pass

        #hse
        try:
            q = 0
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'hse')
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[len(values)+2][q]=1
                q = q + 1
        except:
            pass
    
    
    elif functional_list == ("pbeU", "scan"):
        #pbeU
        h=len(values)+2
        w=len(configurations)+1
        output = [[0 for x in range(w)] for y in range(h)] 
        q = 1
        p = 1
        for x in configurations:
            for i in values:
                directory=os.path.join(dir,"conf_"+str(x), 'pbeU',str(i))
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[p][q]=1
                            output[p][0]=values[p-1]
                p = p + 1
            p = 1
            output[0][q]=q-1
            q = q + 1

        #scan
        try: 
            q = 1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'scan')
                os.chdir(directory)
                with open('out', 'r') as searchfile:
                    for line in searchfile:
                        if 'reached required accuracy - stopping structural energy minimisation' in line:
                            output[len(values)+1][q]=1
                q = q + 1

        except:
            pass
        
        
    return output


In [81]:


import os
from conf_checker import conf_checker

def energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)
    else:
        print("This file doesn't exists")

    if (configuration == True):
        configurations=conf_checker(model,li_concentration,structure)
    else:
        configurations=[conf]

    try:
        h=len(U_value)
        w=len(configurations)+1
        b = [[0 for x in range(w)] for y in range(h)]   
    except:
        h=1
        w=len(configurations)+1
        b = [[0 for x in range(w)] for y in range(h)] 

    try:
        if (len(U_value)>1):
            values = U_value    
    except:
        values = [U_value]

    #pbeU 
    if functional_list == 'pbeU':
        j = 1
        q = 0
        for x in configurations:
            for i in values:
                directory=os.path.join(dir,"conf_"+str(x), 'pbeU',str(i))
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[q][j]=line[-len(U_value)-3:-10]
                            except:
                                b[q][j]=line[-17:-10]
                b[q][0] = i
                q = q + 1
            q = 0
            j = j + 1
    
    #scan
    try:
        if functional_list == 'scan':
            h=1
            w=len(configurations)+1
            b = [[0 for x in range(w)] for y in range(h)]
            j = 1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'scan')
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[0][j]=line[-len(U_value)-3:-10]
                            except:
                                b[0][j]=line[-17:-10]
                j = j + 1
    except:
        pass
    
    #hse
    try:
        if functional_list == 'hse':
            h=1
            w=len(configurations)+1
            b = [[0 for x in range(w)] for y in range(h)]
            j = 1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'hse')
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[0][j]=line[-len(U_value)-3:-10]
                            except:
                                b[0][j]=line[-17:-10]
                j = j + 1
    except:
        pass
    
    #pbeU,scan and hse
    if functional_list == ("pbeU", "scan" , "hse"):
        try:
            h=len(values)+2
            w=len(configurations)+1
            b = [[0 for x in range(w)] for y in range(h)]   
        except:
            h=3
            w=len(configurations)+1
            b = [[0 for x in range(w)] for y in range(h)] 
        
        j = 1
        q = 0
        for x in configurations:
            for i in values:
                directory=os.path.join(dir,"conf_"+str(x), 'pbeU',str(i))
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[q][j]=line[-len(U_value)-3:-10]
                            except:
                                b[q][j]=line[-17:-10]
                b[q][0] = i
                q = q + 1
            q = 0
            j = j + 1
            
        #scan    
        try:  
            j=1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'scan')
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[len(values)][j]=line[-len(U_value)-3:-10]
                            except:
                                b[len(values)][j]=line[-17:-10]
                j = j + 1
        except:
            pass
        
        #hse
        try:
            j=1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'hse')
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[len(values)+1][j]=line[-len(U_value)-3:-10]
                            except:
                                b[len(values)+1][j]=line[-17:-10]
                j = j + 1
        except:
            pass
        
    #pbeU,scan
    if functional_list == ("pbeU", "scan"):
        try:
            h=len(values)+1
            w=len(configurations)+1
            b = [[0 for x in range(w)] for y in range(h)]   
        except:
            h=3
            w=len(configurations)+1
            b = [[0 for x in range(w)] for y in range(h)] 
        
        j = 1
        q = 0
        for x in configurations:
            for i in values:
                directory=os.path.join(dir,"conf_"+str(x), 'pbeU',str(i))
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[q][j]=line[-len(U_value)-3:-10]
                            except:
                                b[q][j]=line[-17:-10]
                b[q][0] = i
                q = q + 1
            q = 0
            j = j + 1
            
        #scan    
        try:  
            j=1
            for x in configurations:
                directory=os.path.join(dir,"conf_"+str(x), 'scan')
                os.chdir(directory)
                with open('OUTCAR', 'r') as searchfile:
                    for line in searchfile:
                        if 'free  en' in line:
                            try:
                                b[len(values)][j]=line[-len(U_value)-3:-10]
                            except:
                                b[len(values)][j]=line[-17:-10]
                j = j + 1
        except:
            pass



    tuple_join = ()
    for i in range(0,len(configurations)):
        lst_mixed = ['E. conf',i, '(eV)']
        str_join = " ".join(str(x) for x in lst_mixed)
        tuple_join = tuple_join + tuple([str_join])   

    header = ("U-value (eV)",) + tuple_join

    #test = tabulate(b,headers=header, tablefmt="latex", floatfmt=(".1f",".2f"))
   
    return (header,b)


In [82]:

# This functions calculates the baseline energy. This is the energy of the fully discharged material.

import os

def baseline_energy(model,li_concentration,structure,functional_list,U_value,conf):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+str(2)+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+str(2)+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+str(2))
        
    h=len(U_value)
    baseline = [0] * h
    j=0
    for i in U_value:
        directory=os.path.join(dir,'conf_'+str(0),functional_list,str(i))
        os.chdir(directory)
        with open('OUTCAR', 'r') as searchfile:
            for line in searchfile:
                if 'free  en' in line:
                    baseline[j]=float(line[-14-3:-10])
        j = j + 1
    return baseline

In [83]:


import os
import pymatgen as pmg

# Gives the magnetization for a specific configuration
def magnetic_check(model,structure,li_concentration,functional_list,U_value,conf):

    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)

    try:
        if (len(U_value)>1):
            values = U_value
            h=len(values)
            if (model == 'Li2MnSnO3'):
                w=4
            else:
                w=3
            results = [[0 for x in range(w)] for y in range(h)] 
            results_test = [[0 for x in range(w)] for y in range(h)] 

    except:
        values = [U_value]
        h=1
        if (model == 'Li2MnSnO3'):
            w=4
        else:
            w=3
        results = [[0 for x in range(w)] for y in range(h)] 
        results_test = [[0 for x in range(w)] for y in range(h)]
    
    
    #pbeU
    if functional_list == 'pbeU':    
        q = 0
        p = 0
        for i in values:
            results[p][0] = i
            results_test[p][0] = i
            p = p + 1


        for u in values:
            directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()
            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values


            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])

        # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
                magnetic_Mn = _sum/count
            results[q][1]= magnetic_Mn

        # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[q][2]=magnetic_Sn
                results[q][3]= magnetic_O
            else:
                results[q][2]= magnetic_O
            q = q + 1


    ## SCAN
    elif functional_list == 'scan':
        h=1
        if (model == 'Li2MnSnO3'):
            w=4
        else:
            w=3
        results = [[0 for x in range(w)] for y in range(h)] 
        results_test = [[0 for x in range(w)] for y in range(h)] 


        #try:
        directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
        os.chdir(directory)

        i = 0
        free_energy_form_different_U_values={}
        with open('OUTCAR','r') as searchfile:
            content = searchfile.readlines()

        with open('OUTCAR','r') as searchfile:
            for line in searchfile:
                    i = i + 1
                    if '# of ion       s       p       d       tot' in line:
                        free_energy_form_different_U_values[i]=line
        free_energy_form_different_U_values

        key = sorted(free_energy_form_different_U_values.keys())[-1]

        s = pmg.Structure.from_file('POSCAR')
        if (model == 'Li2MnSnO3'):
            a=s.formula
            x = int(a[2])
            y = int(a[6:8])
            z = int(a[-2:])
        else:
            a=s.formula
            x = int(a[2])
            y = int(a[6])
            z = int(a[-1:])

        # Calculates the avarage magnetic moment on the second element
        j = 0
        a = 0
        magnetic={}
        for i in range(x,y+x):
            j = j + 1
            mag = content[key+i+1]
            magnetic[j] = mag[-6:-1]

        count = 0
        _sum = 0
        for i in range(1,len(magnetic)):
            count += 1
            _sum += float(magnetic[i])
        magnetic_Mn = _sum/count
        results[0][1]= magnetic_Mn

        # Calculates the avarage magnetic moment on the third element
        j = 0
        a = 0
        magnetic={}
        for i in range(y+x,y+x+z+1):
            j = j + 1
            mag = content[key+i+1]
            magnetic[j] = mag[-7:-1]

        count = 0
        _sum = 0
        for i in range(1,len(magnetic)):
            count += 1
            _sum += float(magnetic[i])
        magnetic_O = _sum/count
        if (model == 'Li2MnSnO3'):
            mag = content[key+x+y+1]
            magnetic_Sn = mag[-6:-1]
            results[0][2]=magnetic_Sn
            results[0][3]= magnetic_O
        else:
            results[0][2]= magnetic_O  

        #except:
            #pass


    ## HSE06
    elif functional_list == 'hse':
        h=1
        if (model == 'Li2MnSnO3'):
            w=4
        else:
            w=3
        results = [[0 for x in range(w)] for y in range(h)] 
        results_test = [[0 for x in range(w)] for y in range(h)] 


        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()

            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values

            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])
            # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_Mn = _sum/count
            results[0][1]= magnetic_Mn

            # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[0][2]=magnetic_Sn
                results[0][3]= magnetic_O
            else:
                results[0][2]= magnetic_O
        except:
            pass
    
    
    elif functional_list == ("pbeU", "scan" , "hse"):
        try:
            if (len(U_value)>1):
                values = U_value
                h=len(values)+2
                if (model == 'Li2MnSnO3'):
                    w=4
                else:
                    w=3
                results = [[0 for x in range(w)] for y in range(h)] 
                results_test = [[0 for x in range(w)] for y in range(h)] 

        except:
            values = [U_value]
            h=1+2
            if (model == 'Li2MnSnO3'):
                w=4
            else:
                w=3
            results = [[0 for x in range(w)] for y in range(h)] 
            results_test = [[0 for x in range(w)] for y in range(h)]
            
        #pbeU   
        q = 0
        p = 0
        for i in values:
            results[p][0] = i
            results_test[p][0] = i
            p = p + 1


        for u in values:
            directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()
            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values


            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])

        # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
                magnetic_Mn = _sum/count
            results[q][1]= magnetic_Mn

        # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[q][2]=magnetic_Sn
                results[q][3]= magnetic_O
            else:
                results[q][2]= magnetic_O
            q = q + 1


    ## SCAN
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()

            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values

            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])

            # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_Mn = _sum/count
            results[q][1]= magnetic_Mn

            # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[q][2]=magnetic_Sn
                results[q][3]= magnetic_O
            else:
                results[q][2]= magnetic_O  

        except:
            pass


    ## HSE06
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()

            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values

            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])
            # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_Mn = _sum/count
            results[q+1][1]= magnetic_Mn

            # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            results[q+1][2]= magnetic_O
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[q+1][2]=magnetic_Sn
                results[q+1][3]= magnetic_O
            else:
                results[q+1][3]= magnetic_O
        except:
            pass
        
        
    elif functional_list == ("pbeU", "scan"):
        try:
            if (len(U_value)>1):
                values = U_value
                h=len(values)+1
                if (model == 'Li2MnSnO3'):
                    w=4
                else:
                    w=3
                results = [[0 for x in range(w)] for y in range(h)] 
                results_test = [[0 for x in range(w)] for y in range(h)] 

        except:
            values = [U_value]
            h=1+1
            if (model == 'Li2MnSnO3'):
                w=4
            else:
                w=3
            results = [[0 for x in range(w)] for y in range(h)] 
            results_test = [[0 for x in range(w)] for y in range(h)]
            
        #pbeU   
        q = 0
        p = 0
        for i in values:
            results[p][0] = i
            results_test[p][0] = i
            p = p + 1


        for u in values:
            directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()
            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values


            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])

        # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
                magnetic_Mn = _sum/count
            results[q][1]= magnetic_Mn

        # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[q][2]= magnetic_Sn
                results[q][3]= magnetic_O
            else:
                results[q][2]= magnetic_O
            q = q + 1


    ## SCAN
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)

            i = 0
            free_energy_form_different_U_values={}
            with open('OUTCAR','r') as searchfile:
                content = searchfile.readlines()

            with open('OUTCAR','r') as searchfile:
                for line in searchfile:
                        i = i + 1
                        if '# of ion       s       p       d       tot' in line:
                            free_energy_form_different_U_values[i]=line
            free_energy_form_different_U_values

            key = sorted(free_energy_form_different_U_values.keys())[-1]

            s = pmg.Structure.from_file('POSCAR')
            if (model == 'Li2MnSnO3'):
                a=s.formula
                x = int(a[2])
                y = int(a[6:8])
                z = int(a[-2:])
            else:
                a=s.formula
                x = int(a[2])
                y = int(a[6])
                z = int(a[-1:])

            # Calculates the avarage magnetic moment on the second element
            j = 0
            a = 0
            magnetic={}
            for i in range(x,y+x):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-6:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_Mn = _sum/count
            results[q][1]= magnetic_Mn

            # Calculates the avarage magnetic moment on the third element
            j = 0
            a = 0
            magnetic={}
            for i in range(y+x,y+x+z+1):
                j = j + 1
                mag = content[key+i+1]
                magnetic[j] = mag[-7:-1]

            count = 0
            _sum = 0
            for i in range(1,len(magnetic)):
                count += 1
                _sum += float(magnetic[i])
            magnetic_O = _sum/count
            if (model == 'Li2MnSnO3'):
                mag = content[key+x+y+1]
                magnetic_Sn = mag[-6:-1]
                results[q][2]= magnetic_Sn
                results[q][3]= magnetic_O
            else:
                results[q][2]= magnetic_O 

        except:
            pass
        
    if model == 'Li2MnO3':
        header = ("U-value (eV)", 'Mn (\mu_B)', 'O (\mu_B)')
    elif model == 'Li2IrO3':
        header = ("U-value (eV)", 'Ir (\mu_B)', 'O (\mu_B)')
    elif model == 'Li2MnSnO3':
        header = ("U-value (eV)", 'Mn (\mu_B)', 'Sn (\mu_B)', 'O (\mu_B)')
        
    #return (tabulate(results, headers=header, tablefmt="latex", floatfmt=".2f"),results)
    return (header,results)


In [84]:
# %load baseline_energy.py
# This functions calculates the baseline energy. This is the energy of the fully discharged material.

import os

def baseline_energy(model,li_concentration,structure,functional_list,U_value,conf):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+str(2)+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+str(2)+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+str(2))
        
    h=len(U_value)
    baseline = [0] * h
    j=0
    for i in U_value:
        directory=os.path.join(dir,'conf_'+str(0),functional_list,str(i))
        os.chdir(directory)
        with open('OUTCAR', 'r') as searchfile:
            for line in searchfile:
                if 'free  en' in line:
                    baseline[j]=float(line[-14-3:-10])
        j = j + 1
    return baseline

In [85]:


import os
import matplotlib.pyplot as plt
from conf_checker import conf_checker
from baseline_energy import baseline_energy
from energy_check import energy_check

def visualization_energy_difference(model,li_concentration,structure,functional_list,U_value,conf,configuration,energy_baseline):

    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)

    directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(U_value)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06   
    fig=plt.figure()
    ax = plt.subplot(111)
    if functional_list == 'pbeU':
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        #fig=plt.figure()
        #ax = plt.subplot(111)

        if (configuration == True):
            configurations=conf_checker(model,li_concentration,structure)
            if len(configurations) > 10:
                configurations = configurations[0:-1]

            w=14
            h=len(configurations)
            energy = [[0 for x in range(w)] for y in range(h)] 

            try:
                for j in range(0,len(configurations)):
                    q=0
                    for a in values:
                        directory=os.path.join(dir,"conf_"+str(j), 'pbeU',str(a))
                        os.chdir(directory)
                        with open('OUTCAR', 'r') as searchfile:
                            for line in searchfile:
                                if 'free  en' in line:
                                    try:
                                        if float(line[-len(values)-3:-10]) > -100:
                                            energy[j][q]=float(line[-len(values)-3:-9])
                                        else:
                                            energy[j][q]=float(line[-len(values)-3:-10])
                                    except:
                                        if float(line[-len(values)-3:-10]) > -100:
                                            energy[j][q]=float(line[-16:-10])
                                        else:
                                            energy[j][q]=float(line[-17:-10])
                        q = q + 1
            except:
                pass
    
            if (energy_baseline == 'Absolute'):
                energy_new = energy
            elif (energy_baseline == 'Minimum'):
                w=14
                h=len(configurations)
                energy_new = [[0 for x in range(w)] for y in range(h)] 
                energy_min=[0] * h
                energy_min_label = [0] * len(values)

                for j in range(0,len(values)):
                    energy_min=[0] * h
                    for i in range (0,len(configurations)):
                        energy_min[i]=energy[i][j]
                    energy_min_label[j] = min(energy_min)

                for j in range(0,h):
                    for i in range (0,len(values)):
                        energy_new[j][i] = float(energy[j][i]) - float(energy_min_label[i])
                
            elif (energy_baseline == 'Discharged'):
                w=14
                h=len(configurations)
                energy_new = [[0 for x in range(w)] for y in range(h)] 
                energy_min=[0] * h
                energy_min_label = [0] * len(values)
                
                baseline = baseline_energy(model,li_concentration,structure,functional_list,U_value,conf)
                for j in range(0,len(values)):
                    for i in range (0,len(configurations)):
                        energy_new[i][j] = energy[i][j]-baseline[j]
                        #energy_new[i][j] = energy[i][j]-baseline_energy(model,li_concentration,structure,U_value,conf)[j]

            for x in range(0,len(configurations)):
                i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
                ax.plot(i, energy_new[x], label='Conf_'+str(x),linestyle='--',marker='o')

            ax.legend()
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            plt.xticks(i, my_xticks)
            test = ax.legend(loc='upper center', bbox_to_anchor=(1.2, 1), shadow=True, ncol=1)
            plt.xlabel('U-value (eV)')
            plt.ylabel('Energy difference (eV)')
            plt.title('Energy difference for different configurations and U-values')
            plt.show()

        else:
            configurations=conf
            w=14
            energy = [0]*w
            energy_new = [0]*w

            try:
                q=0
                for a in values:
                    directory=os.path.join(dir,"conf_"+str(configurations), 'pbeU',str(a))
                    os.chdir(directory)
                    with open('OUTCAR', 'r') as searchfile:
                        for line in searchfile:
                            if 'free  en' in line:
                                try:
                                    if float(line[-len(values)-3:-10]) > -100:
                                        energy[q]=float(line[-len(values)-3:-9])
                                    else:
                                        energy[q]=float(line[-len(values)-3:-10])
                                except:
                                    if float(line[-len(values)-3:-10]) > -100:
                                        energy[q]=float(line[-16:-10])
                                    else:
                                        energy[q]=float(line[-17:-10])
                    q = q + 1  
                q=0
                energy_max=max(energy)
                for i in values:
                    energy_new[q]=float(energy_max)-float(energy[q])
                    q=q+1
                    
                if (energy_baseline == 'Absolute'):
                    energy_new = energy
                elif (energy_baseline == 'Minimum'):
                    q=0
                    energy_max=max(energy)
                    for i in values:
                        energy_new[q]=float(energy_max)-float(energy[q])
                        q=q+1

                elif (energy_baseline == 'Discharged'):
                    baseline = baseline_energy(model,li_concentration,structure,functional_list,U_value,conf)
                    q=0
                    for i in values:
                        energy_new[q]=float(energy[q])-float(baseline[q])
                        q=q+1 
            except:
                pass

            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            ax.plot(i, energy_new, label='Conf_'+str(configurations),linestyle='--',marker='o')

            ax.legend()
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            plt.xticks(i, my_xticks)
            test = ax.legend(loc='upper center', bbox_to_anchor=(1.2, 1), shadow=True, ncol=1)
            plt.xlabel('U-value (eV)')
            plt.ylabel('Energy difference (eV)')
            plt.title('Energy difference for configuration '+str(configurations)+' with different U-values')
            plt.show()

        os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
        fig.savefig("Test.pdf",bbox_inches='tight',pad_inches=1)
        
    elif functional_list == 'scan':
        if (configuration == True):
            configurations=conf_checker(model,li_concentration,structure)
            if len(configurations) > 10:
                configurations = configurations[0:-1]
        else:
            configurations=conf
                
        plotting = [0] * (len(configurations))
        plotting_new = [0] * (len(configurations))
        [header,energy] = energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        q=1
        for i in configurations:
            plotting[i]=float(energy[0][i+1])
            q = q + 1
        
        if (energy_baseline == 'Absolute'):
            plotting_new = plotting
        elif (energy_baseline == 'Minimum'):
            plotting_max=max(plotting)
            for i in configurations:
                plotting_new[i]=plotting_max-plotting[i]

        elif (energy_baseline == 'Discharged'):
            baseline = baseline_energy(model,li_concentration,structure,functional_list,U_value,conf)
            for i in range (0,len(configurations)):
                plotting_new[i] = plotting[i]-baseline
                #energy_new[i][j] = energy[i][j]-baseline_energy(model,li_concentration,structure,U_value,conf)[j]

        plt.plot(configurations,plotting_new,linestyle='--',marker='o')
        my_xticks=()
        for i in configurations:
            a = tuple(['c_'+str(i)])
            my_xticks = my_xticks + a
        plt.xticks(configurations, my_xticks)
        plt.xlabel('Configurations')
        plt.ylabel('Energy difference (eV)')
        plt.title('Energy difference for configurations calculated with SCAN')
        plt.show()
    

    elif functional_list == 'hse':
        if (configuration == True):
            configurations=conf_checker(model,li_concentration,structure)
            if len(configurations) > 10:
                configurations = configurations[0:-1]
        else:
            configurations=conf
                
        plotting = [0] * (len(configurations))
        plotting_new = [0] * (len(configurations))
        [header,energy] = energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        q=1
        for i in configurations:
            plotting[i]=float(energy[0][i+1])
            q = q + 1
        
        if (energy_baseline == 'Absolute'):
            plotting_new = plotting
        elif (energy_baseline == 'Minimum'):
            plotting_max=max(plotting)
            for i in configurations:
                plotting_new[i]=plotting_max-plotting[i]

        elif (energy_baseline == 'Discharged'):
            baseline = baseline_energy(model,li_concentration,structure,functional_list,U_value,conf)
            for i in range (0,len(configurations)):
                plotting_new[i] = plotting[i]-baseline
                #energy_new[i][j] = energy[i][j]-baseline_energy(model,li_conc
    
    
        plt.plot(configurations,plotting_new,linestyle='--',marker='o')
        my_xticks=()
        for i in configurations:
            a = tuple(['c_'+str(i)])
            my_xticks = my_xticks + a
        plt.xticks(configurations, my_xticks)
        plt.xlabel('Configurations')
        plt.ylabel('Energy difference (eV)')
        plt.title('Energy difference for configurations calculated with HSE06')
        plt.show()
        
    elif functional_list == ("pbeU", "scan" , "hse"):
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        #fig=plt.figure()
        #ax = plt.subplot(111)

        if (configuration == True):
            configurations=conf_checker(model,li_concentration,structure)
            if len(configurations) > 10:
                configurations = configurations[0:-1]

            w=14
            h=len(configurations)
            energy = [[0 for x in range(w)] for y in range(h)] 

            try:
                for j in range(0,len(configurations)):
                    q=0
                    for a in values:
                        directory=os.path.join(dir,"conf_"+str(j), 'pbeU',str(a))
                        os.chdir(directory)
                        with open('OUTCAR', 'r') as searchfile:
                            for line in searchfile:
                                if 'free  en' in line:
                                    try:
                                        if float(line[-len(values)-3:-10]) > -100:
                                            energy[j][q]=float(line[-len(values)-3:-9])
                                        else:
                                            energy[j][q]=float(line[-len(values)-3:-10])
                                    except:
                                        if float(line[-len(values)-3:-10]) > -100:
                                            energy[j][q]=float(line[-16:-10])
                                        else:
                                            energy[j][q]=float(line[-17:-10])
                        q = q + 1
            except:
                pass
            #pbeu
            w=16
            h=len(configurations)
            energy_new = [[0 for x in range(w)] for y in range(h)] 
            energy_min=[0] * h
            energy_min_label = [0] * len(values)

            for j in range(0,len(values)):
                energy_min=[0] * h
                for i in range (0,len(configurations)):
                    energy_min[i]=energy[i][j]
                energy_min_label[j] = max(energy_min)

            for j in range(0,h):
                for i in range (0,len(values)):
                    energy_new[j][i] = float(energy[j][i]) - float(energy_min_label[i])
            
            #scan
            plotting = [0] * (len(configurations))
            plotting_new = [0] * (len(configurations))
            [header,energy] = energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration)
            q=1
            for i in configurations:
                plotting[i]=float(energy[-2][i+1])
                q = q + 1
            plotting_max=max(plotting)    
            for j in configurations:
                energy_new[j][-2]=plotting_max-plotting[j]
            
            #hse
            for i in configurations:
                plotting[i]=float(energy[-1][i+1])
                q = q + 1
            plotting_max=max(plotting)    
            for j in configurations:
                energy_new[j][-1]=plotting_max-plotting[j]
            
            
            
            for x in range(0,len(configurations)):
                i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]
                ax.plot(i, energy_new[x], label='Conf_'+str(x),linestyle='--',marker='o')

            ax.legend()
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan','hse')
            plt.xticks(i, my_xticks)
            test = ax.legend(loc='upper center', bbox_to_anchor=(1.2, 1), shadow=True, ncol=1)
            plt.xlabel('U-value (eV)')
            plt.ylabel('Energy difference (eV)')
            plt.title('Energy difference for different configurations and U-values')
            plt.show()
        
    elif functional_list == ("pbeU", "scan"):
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        #fig=plt.figure()
        #ax = plt.subplot(111)

        if (configuration == True):
            configurations=conf_checker(model,li_concentration,structure)
            if len(configurations) > 10:
                configurations = configurations[0:-1]

            w=14
            h=len(configurations)
            energy = [[0 for x in range(w)] for y in range(h)] 

            try:
                for j in range(0,len(configurations)):
                    q=0
                    for a in values:
                        directory=os.path.join(dir,"conf_"+str(j), 'pbeU',str(a))
                        os.chdir(directory)
                        with open('OUTCAR', 'r') as searchfile:
                            for line in searchfile:
                                if 'free  en' in line:
                                    try:
                                        if float(line[-len(values)-3:-10]) > -100:
                                            energy[j][q]=float(line[-len(values)-3:-9])
                                        else:
                                            energy[j][q]=float(line[-len(values)-3:-10])
                                    except:
                                        if float(line[-len(values)-3:-10]) > -100:
                                            energy[j][q]=float(line[-16:-10])
                                        else:
                                            energy[j][q]=float(line[-17:-10])
                        q = q + 1
            except:
                pass
            
            #pbeu
            w=15
            h=len(configurations)
            energy_new = [[0 for x in range(w)] for y in range(h)] 
            energy_min=[0] * h
            energy_min_label = [0] * len(values)

            for j in range(0,len(values)):
                energy_min=[0] * h
                for i in range (0,len(configurations)):
                    energy_min[i]=energy[i][j]
                energy_min_label[j] = min(energy_min)

            for j in range(0,h):
                for i in range (0,len(values)):
                    energy_new[j][i] = float(energy[j][i]) - float(energy_min_label[i])
            
            #scan
            plotting = [0] * (len(configurations))
            plotting_new = [0] * (len(configurations))
            [header,energy] = energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration)
            q=1
            for i in configurations:
                plotting[i]=float(energy[-2][i+1])
                q = q + 1
            plotting_max=max(plotting)    
            for j in configurations:
                energy_new[j][-1]=plotting_max-plotting[j]
            
            for x in range(0,len(configurations)):
                i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
                ax.plot(i, energy_new[x], label='Conf_'+str(x),linestyle='--',marker='o')

            ax.legend()
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            plt.xticks(i, my_xticks)
            test = ax.legend(loc='upper center', bbox_to_anchor=(1.2, 1), shadow=True, ncol=1)
            plt.xlabel('U-value (eV)')
            plt.ylabel('Energy difference (eV)')
            plt.title('Energy difference for different configurations and U-values')
            plt.show()

        else:
            print('This figure is only interesting for multiple configurations')
    
    os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
    fig.savefig("Test.pdf",bbox_inches='tight',pad_inches=1)

In [86]:


import os
import matplotlib.pyplot as plt
from lattice_check import lattice_check
from lattice_angles_check import lattice_angles_check

def visualization_lattice(model,li_concentration,structure,functional_list,U_value,conf): 
    
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)
    #try:
    if (len(U_value)>1):
        values = U_value 
        
    fig=plt.figure()

    if functional_list == 'pbeU':
        a_angle_a=[0] * (len(values))
        a_angle_b=[0] * (len(values))
        a_angle_c=[0] * (len(values))
        a_lattice_b=[0] * (len(values))
        a_lattice_a=[0] * (len(values))
        a_lattice_c=[0] * (len(values))
        j = 0 
        
        if (model == 'Li2MnO3'):
            for i in values:
                a_lattice_c[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]

                a_angle_a[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

                a_lattice_b[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][2]

                a_angle_c[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
                j=j+1
            
        elif (model == 'Li2IrO3'):
            for i in values:
                a_lattice_c[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]

                a_angle_a[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

                a_lattice_a[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

                a_angle_c[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
                j=j+1

        i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
        
        if (model == 'Li2MnO3'):
            plt.subplot(2, 1, 1)
            plt.plot(i, a_lattice_c, 'b-',label='lattice c-parameters',linestyle='--',marker='o', color='b')
            plt.plot(i, a_lattice_b, 'g-',label='lattice a/b-parameters',linestyle='--',marker='o', color='r')
            plt.ylabel('Lattice parameters (A)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            plt.xticks(i, my_xticks)
            plt.title('Lattice parameters and angles for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)

            plt.subplot(2, 1, 2)
            plt.plot(i, a_angle_a, 'r-',label='lattice alfa/beta-angles',linestyle='--',marker='o', color='r')
            plt.plot(i, a_angle_c, 'y-',label='lattice gamma-angles',linestyle='--',marker='o', color='b')
            plt.xlabel('U-value (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            plt.xticks(i, my_xticks)
            plt.ylabel('Angles (°)')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=True, ncol=1)
            
        elif (model == 'Li2IrO3'):
            plt.subplot(2, 1, 1)
            plt.plot(i, a_lattice_a, 'b-',label='lattice a-parameters',linestyle='--',marker='o', color='b')
            plt.plot(i, a_lattice_c, 'g-',label='lattice b/c-parameters',linestyle='--',marker='o', color='r')
            plt.ylabel('Lattice parameters (A)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            plt.xticks(i, my_xticks)
            plt.title('Lattice parameters and angles for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)

            plt.subplot(2, 1, 2)
            plt.plot(i, a_angle_a, 'r-',label='lattice alpha-angles',linestyle='--',marker='o', color='r')
            plt.plot(i, a_angle_c, 'y-',label='lattice beta/gamma-angles',linestyle='--',marker='o', color='b')
            plt.xlabel('U-value (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            plt.xticks(i, my_xticks)
            plt.ylabel('Angles (°)')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=True, ncol=1)
        
        plt.show()
    
    elif functional_list == ("pbeU", "scan" , "hse"):

        a_angle_a=[0] * (len(values)+2)
        a_angle_b=[0] * (len(values)+2)
        a_angle_c=[0] * (len(values)+2)
        a_lattice_b=[0] * (len(values)+2)
        a_lattice_a=[0] * (len(values)+2)
        a_lattice_c=[0] * (len(values)+2)
        j = 0 

        for i in range(len(values)+2):
            a_lattice_c[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]

            a_angle_a[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

            a_lattice_b[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][2]

            a_angle_c[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
            j=j+1

        i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]
      
        plt.subplot(2, 1, 1)
        plt.plot(i, a_lattice_c, 'b-',label='lattice c-parameters',linestyle='--',marker='o', color='b')
        plt.plot(i, a_lattice_b, 'g-',label='lattice a/b-parameters',linestyle='--',marker='o', color='r')
        plt.ylabel('Lattice parameters (A)')
        my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan','hse')
        i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]
        plt.xticks(i, my_xticks)
        plt.title('Lattice parameters and angles for different U-values')
        plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)

        plt.subplot(2, 1, 2)
        plt.plot(i, a_angle_a, 'r-',label='lattice alfa/beta-angles',linestyle='--',marker='o', color='r')
        plt.plot(i, a_angle_c, 'y-',label='lattice gamma-angles',linestyle='--',marker='o', color='b')
        plt.xlabel('U-value (eV)')
        my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan','hse')
        i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]
        plt.xticks(i, my_xticks)
        plt.ylabel('Angles (°)')
        plt.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=True, ncol=1)

        plt.show()
        
    elif functional_list == ("pbeU", "scan"):

        a_angle_a=[0] * (len(values)+1)
        a_angle_b=[0] * (len(values)+1)
        a_angle_c=[0] * (len(values)+1)
        a_lattice_b=[0] * (len(values)+1)
        a_lattice_a=[0] * (len(values)+1)
        a_lattice_c=[0] * (len(values)+1)
        j = 0 
        
        if (model == 'Li2MnO3'):
            for i in range(len(values)+1):
                a_lattice_c[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]

                a_angle_a[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

                a_lattice_b[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][2]

                a_angle_c[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
                j=j+1
            
        elif (model == 'Li2IrO3'):
            for i in range(len(values)+1):
                a_lattice_c[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]

                a_angle_a[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

                a_lattice_a[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]

                a_angle_c[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
                j=j+1
                
        elif (model == 'Li2MnSnO3'):
            for i in range(len(values)+1):
                a_lattice_a[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]
                a_lattice_b[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][2]
                a_lattice_c[j] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
                
                a_angle_a[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][1]
                a_angle_b[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][2]
                a_angle_c[j] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)[1][j][3]
                j=j+1

        i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
        
        if (model == 'Li2MnO3'):
            plt.subplot(2, 1, 1)
            plt.plot(i, a_lattice_c, 'b-',label='lattice a-parameters',linestyle='--',marker='o', color='b')
            plt.plot(i, a_lattice_b, 'g-',label='lattice b/c-parameters',linestyle='--',marker='o', color='r')
            plt.ylabel('Lattice parameters (A)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.title('Lattice parameters and angles for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)

            plt.subplot(2, 1, 2)
            plt.plot(i, a_angle_a, 'r-',label='lattice alfa/beta-angles',linestyle='--',marker='o', color='r')
            plt.plot(i, a_angle_c, 'y-',label='lattice gamma-angles',linestyle='--',marker='o', color='b')
            plt.xlabel('U-value (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.ylabel('Angles (°)')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=True, ncol=1)

        elif (model == 'Li2IrO3'):
            plt.subplot(2, 1, 1)
            plt.plot(i, a_lattice_c, 'b-',label='lattice a-parameters',linestyle='--',marker='o', color='b')
            plt.plot(i, a_lattice_a, 'g-',label='lattice b/c-parameters',linestyle='--',marker='o', color='r')
            plt.ylabel('Lattice parameters (A)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.title('Lattice parameters and angles for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)

            plt.subplot(2, 1, 2)
            plt.plot(i, a_angle_a, 'r-',label='lattice alfa-angles',linestyle='--',marker='o', color='r')
            plt.plot(i, a_angle_c, 'y-',label='lattice beta/gamma-angles',linestyle='--',marker='o', color='b')
            plt.xlabel('U-value (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.ylabel('Angles (°)')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=True, ncol=1)
            
        elif (model == 'Li2MnSnO3'):
            plt.subplot(2, 1, 1)
            plt.plot(i, a_lattice_a,label='lattice a/b-parameters',linestyle='--',marker='o', color='b')
            #plt.plot(i, a_lattice_b,label='lattice b-parameters',linestyle='--',marker='o', color='r')
            plt.plot(i, a_lattice_c,label='lattice c-parameters',linestyle='--',marker='o', color='r')
            plt.ylabel('Lattice parameters (A)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.title('Lattice parameters and angles for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)

            plt.subplot(2, 1, 2)
            plt.plot(i, a_angle_a,label='lattice alfa/beta-angles',linestyle='--',marker='o', color='b')
            #plt.plot(i, a_angle_b,label='lattice beta-angles',linestyle='--',marker='o', color='b')
            plt.plot(i, a_angle_c,label='lattice gamma-angles',linestyle='--',marker='o', color='r')
            plt.xlabel('U-value (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.ylabel('Angles (°)')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 0.8), shadow=True, ncol=1)
            
        plt.show()

    else:
        print('Not yet implemented. (visualization_lattice.py)')
        
    os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
    fig.savefig("Test.pdf",bbox_inches='tight',pad_inches=1)


In [87]:

# Getting lattice parameters
import os
import pymatgen as pmg

def lattice_check(model,structure,li_concentration,functional_list,U_value,conf):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)
    
    try:
        if (len(U_value)>1):
            values = U_value    
    except:
        values = [U_value]
    
    if functional_list == 'pbeU':
            
        h=len(values)
        w=4
        lattice_parameters_length = [[0 for x in range(w)] for y in range(h)]

        for i in range(0,3):
            q=0
            for u in values:
                directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
                os.chdir(directory)
                s = pmg.Structure.from_file('CONTCAR')
                lattice_parameters_length[q][i+1]=s.lattice.abc[i]
                lattice_parameters_length[q][0]=u
                q = q + 1
        
    elif functional_list == 'scan':
        h=1
        w=4
        lattice_parameters_length = [[0 for x in range(w)] for y in range(h)]
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_length[0][i+1]=s.lattice.abc[i]
        except:
            pass
            
    elif functional_list == 'hse':
        h=1
        w=4
        lattice_parameters_length = [[0 for x in range(w)] for y in range(h)]
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory) 
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_length[0][i+1]=s.lattice.abc[i]
        except:
            pass
        
    elif functional_list == ("pbeU", "scan" , "hse"):
        h=len(values)+2
        w=4
        lattice_parameters_length = [[0 for x in range(w)] for y in range(h)]
    #pbeu part
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        for i in range(0,3):
            q=0
            for u in values:
                directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
                os.chdir(directory)
                s = pmg.Structure.from_file('CONTCAR')
                lattice_parameters_length[q][i+1]=s.lattice.abc[i]
                lattice_parameters_length[q][0]=u
                q = q + 1
        try:     
        #scan part
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_length[q][i+1]=s.lattice.abc[i]
        except:
            pass

        #hse part  
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_length[q+1][i+1]=s.lattice.abc[i]       
        except:
            pass
    
    elif functional_list == ("pbeU", "scan"):
        h=len(values)+1
        w=4
        lattice_parameters_length = [[0 for x in range(w)] for y in range(h)]
    #pbeu part
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        for i in range(0,3):
            q=0
            for u in values:
                directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
                os.chdir(directory)
                s = pmg.Structure.from_file('CONTCAR')
                lattice_parameters_length[q][i+1]=s.lattice.abc[i]
                lattice_parameters_length[q][0]=u
                q = q + 1
        try:     
        #scan part
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_length[q][i+1]=s.lattice.abc[i]
        except:
            pass
    
    else:
        pass
    
    header = ("U-value (eV)", 'a-lattice (A)', 'b-lattice (A)','c-lattice (A)')
    
    return (header,lattice_parameters_length)
    #return tabulate(lattice_parameters_length, headers=header, tablefmt="latex", floatfmt=(".1f",".5f"))


In [88]:

# Getting lattice parameters
import os
import pymatgen as pmg

def lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf):
    try:
        if (model == 'Li2MnO3'):
            dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
        elif (model == 'Li2IrO3'):
            dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
        elif (model == 'Li2MnSnO3'):
            dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)
    except:
        print("This file doesn't exist")
    
    try:
        if (len(U_value)>1):
            values = U_value    
    except:
        values = [U_value]
    
    if functional_list == 'pbeU':
            
        h=len(values)
        w=4
        lattice_parameters_angles = [[0 for x in range(w)] for y in range(h)]
  
        try:
            for i in range(0,3):
                q=0
                for u in values:
                    directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
                    os.chdir(directory)
                    s = pmg.Structure.from_file('CONTCAR')
                    lattice_parameters_angles[q][i+1]=s.lattice.angles[i]
                    lattice_parameters_angles[q][0]=u
                    q = q + 1
        except:
            lattice_parameters_angles='File does not exist'
        
    elif functional_list == 'scan':
        h=1
        w=4
        lattice_parameters_angles = [[0 for x in range(w)] for y in range(h)]
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_angles[0][i+1]=s.lattice.angles[i]
        except:
            pass
            
    elif functional_list == 'hse':
        h=1
        w=4
        lattice_parameters_angles = [[0 for x in range(w)] for y in range(h)]
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory) 
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_angles[0][i+1]=s.lattice.angles[i]
        except:
            pass
        
    elif functional_list == ("pbeU", "scan" , "hse"):
        h=len(values)+2
        w=4
        lattice_parameters_angles = [[0 for x in range(w)] for y in range(h)]
    #pbeu part
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        for i in range(0,3):
            q=0
            for u in values:
                directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
                os.chdir(directory)
                s = pmg.Structure.from_file('CONTCAR')
                lattice_parameters_angles[q][i+1]=s.lattice.angles[i]
                lattice_parameters_angles[q][0]=u
                q = q + 1
        try:     
        #scan part
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_angles[q][i+1]=s.lattice.angles[i]
        except:
            pass

        #hse part  
        try:
            directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_angles[q+1][i+1]=s.lattice.angles[i]       
        except:
            pass
        
    elif functional_list == ("pbeU", "scan"):
        h=len(values)+1
        w=4
        lattice_parameters_angles = [[0 for x in range(w)] for y in range(h)]
    #pbeu part
        try:
            if (len(U_value)>1):
                values = U_value    
        except:
            values = [U_value]

        for i in range(0,3):
            q=0
            for u in values:
                directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(u)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
                os.chdir(directory)
                s = pmg.Structure.from_file('CONTCAR')
                lattice_parameters_angles[q][i+1]=s.lattice.angles[i]
                lattice_parameters_angles[q][0]=u
                q = q + 1
        try:     
        #scan part
            directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06
            os.chdir(directory)
            s = pmg.Structure.from_file('CONTCAR')
            for i in range(0,3):
                lattice_parameters_angles[q][i+1]=s.lattice.angles[i]
        except:
            pass
    
    else:
        pass
    
    header = ("U-value (eV)", 'alfa-angle (°)', 'beta-angle (°)','gamma-angle (°)')
    
    return (header,lattice_parameters_angles)
    #return tabulate(lattice_parameters_angles, headers=header, tablefmt="latex", floatfmt=(".1f",".5f"))

In [89]:


import os
import pymatgen as pmg
from show_structure import show_structure

def visualization(model,li_concentration,structure,functional_list,U_value,conf):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)

    if functional_list == 'pbeU':
        directory=os.path.join(dir,"conf_"+str(conf), 'pbeU',str(U_value)) # <--- Specify U-vaule (last number) or dir for SCAN and HSE06   
    elif functional_list == 'scan':
        directory=os.path.join(dir,"conf_"+str(conf), 'scan') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06   
    elif functional_list == 'hse':
        directory=os.path.join(dir,"conf_"+str(conf), 'hse') # <--- Specify U-vaule (last number) or dir for SCAN and HSE06   


    file=os.path.join(directory, "CONTCAR") # Which file do you want to visualize?
    s = pmg.Structure.from_file(file)
    if (model == 'Li2MnSnO3'):
        show_structure(s,)
    else: 
        show_structure(s,[2,2,2])

In [90]:

import os
import imolecule

def show_structure(structure, supercell=None):
    if supercell:
        structure = structure.copy()
        structure.make_supercell(supercell)
        
    os.chdir('/scratch/antwerpen/204/vsc20412/temp_files') # Make a directory in your scratch for all the temp files.
    structure.to("cif", "temp.cif")
    imolecule.draw("temp.cif")


In [91]:


import os
from magnetic_check import magnetic_check
import matplotlib.pyplot as plt

def magnetic_plot(model,li_concentration,structure,functional_list,U_value,conf):
    
    if functional_list == 'pbeU':
        if (len(U_value)>1):
            values = U_value  

            [header,results] = magnetic_check(model,structure,li_concentration,functional_list,U_value,conf) 

            testing_Mn = [0]*len(values)
            testing_O = [0]*len(values)
            q=0
            for i in values:
                testing_Mn[q] = results[q][1]
                testing_O[q] = results[q][2]
                q = q + 1

            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            fig=plt.figure()
            plt.subplot(2, 1, 1)
            plt.plot(i, testing_Mn, 'b-',label='Magnetic moment '+str(model[3:5]),linestyle='--',marker='o', color='b')
            plt.plot(i, testing_O, 'g-',label='Magnetic moment O',linestyle='--',marker='o', color='r')
            plt.ylabel('Magnetic moment (\mu_B)')
            plt.xlabel('U-values (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            plt.xticks(i, my_xticks)
            plt.title('Magnetic moments for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)
            plt.show()

            os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
            fig.savefig("Test5.pdf",bbox_inches='tight',pad_inches=1)
        else:
            print('oeps')
            
    elif functional_list == ("pbeU", "scan" , "hse"):
        if (len(U_value)>1):
            values = U_value  

            [header,results] = magnetic_check(model,structure,li_concentration,functional_list,U_value,conf) 

            testing_Mn = [0]*(len(values)+2)
            testing_O = [0]*(len(values)+2)
            q=0
            for i in range(0,len(values)+2):
                testing_Mn[q] = results[q][1]
                testing_O[q] = results[q][2]
                q = q + 1

            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]
            fig=plt.figure()
            plt.subplot(2, 1, 1)
            plt.plot(i, testing_Mn, 'b-',label='Magnetic moment '+str(model[3:5]),linestyle='--',marker='o', color='b')
            plt.plot(i, testing_O, 'g-',label='Magnetic moment O',linestyle='--',marker='o', color='r')
            plt.ylabel('Magnetic moment (\mu_B)')
            plt.xlabel('U-values (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan',"hse")
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14,15]
            plt.xticks(i, my_xticks)
            plt.title('Magnetic moments for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)
            plt.show()

            os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
            fig.savefig("Test5.pdf",bbox_inches='tight',pad_inches=1)
        else:
            print('oeps')
    
    elif functional_list == ("pbeU", "scan"):
        if (len(U_value)>1):
            values = U_value  

            [header,results] = magnetic_check(model,structure,li_concentration,functional_list,U_value,conf) 

            testing_Mn = [0]*(len(values)+1)
            testing_O = [0]*(len(values)+1)
            q=0
            for i in range(0,len(values)+1):
                testing_Mn[q] = results[q][1]
                testing_O[q] = results[q][2]
                q = q + 1

            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            fig=plt.figure()
            plt.subplot(2, 1, 1)
            plt.plot(i, testing_Mn, 'b-',label='Magnetic moment '+str(model[3:5]),linestyle='--',marker='o', color='b')
            plt.plot(i, testing_O, 'g-',label='Magnetic moment O',linestyle='--',marker='o', color='r')
            plt.ylabel('Magnetic moment (\mu_B)')
            plt.xlabel('U-values (eV)')
            my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6,'scan')
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14]
            plt.xticks(i, my_xticks)
            plt.title('Magnetic moments for different U-values')
            plt.legend(loc='upper center', bbox_to_anchor=(1.4, 1), shadow=True, ncol=1)
            plt.show()

            os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
            fig.savefig("Test5.pdf",bbox_inches='tight',pad_inches=1)
        else:
            print('oeps')
    else:
        print('Not yet implemented. (magnetic_plot.py)')
    
    os.chdir('/scratch/antwerpen/204/vsc20412/temp_files')
    fig.savefig("Test.pdf",bbox_inches='tight',pad_inches=1)

In [92]:


from energy_check import energy_check

def voltage(model,structure,li_concentration,functional_list,U_value,conf,configuration):
    
    try:
        if (len(U_value)>1):
            values = U_value    
    except:
        values = [U_value]
        
    if structure == 'conv':
        if (len(values)>1):
            h=len(values)
            w=2
            result = [[0 for x in range(w)] for y in range(h)] 
            q=0
            for i in values:
                [header,energy]=energy_check(model,structure,'0.5',functional_list,i,conf,False)
                b = float(energy[0][1])

                [header,energy]=energy_check(model,structure,'2',functional_list,i,'0',False)
                a = float(energy[0][1])

                result[q][1] = (a/4-b/4+1.9*1.5)/1.5
                result[q][0] = i
                q = q + 1
        else:
            result = [0]*2
            
            [header,energy]=energy_check(model,structure,'0.5',functional_list,U_value,conf,False)
            b = float(energy[0][1])

            [header,energy]=energy_check(model,structure,'2',functional_list,U_value,'0',False)
            a = float(energy[0][1])

            result[1] = (a/4-b/4+1.9*1.5)/1.5
            result[0] = float(U_value)
            result = [result]
    else:
        if (len(values)>1):
            h=len(values)
            w=2
            result = [[0 for x in range(w)] for y in range(h)] 
            q=0
            for i in values:
                [header,energy]=energy_check(model,structure,'0.5',functional_list,i,conf,False)
                b = float(energy[0][1])

                [header,energy]=energy_check(model,structure,'2',functional_list,i,'0',False)
                a = float(energy[0][1])

                result[q][1] = (a/2-b/2+1.9*1.5)/1.5
                result[q][0] = i
                q = q + 1
        else:
            result = [0]*2
            [header,energy]=energy_check(model,structure,'0.5',functional_list,U_value,conf,False)
            b = float(energy[0][1])

            [header,energy]=energy_check(model,structure,'2',functional_list,U_value,'0',False)
            a = float(energy[0][1])

            result[1] = (a/2-b/2+1.9*1.5)/1.5
            result[0] = float(U_value)
            result = [result]
            
    header = ("U-value (eV)", 'Voltage (V)')
    return (header,result)

In [93]:


from voltage import voltage
from energy_check import energy_check
from conf_checker import conf_checker
import matplotlib.pyplot as plt


def voltage_visualization(model,structure,li_concentration,functional_list,U_value,conf,configuration):
    fig=plt.figure()
    ax = plt.subplot(111)

    if (configuration == True):
        configurations=conf_checker(model,li_concentration,structure)
    else:
        configurations=[conf]

    try:
        if (len(U_value)>1):
            values = U_value    
    except:
        values = [U_value]

    if (configuration == True):
        w=len(values)
        h=len(configurations)
        plot = [[0 for x in range(w)] for y in range(h)] 
        q = 0
        p = 0
        for j in configurations:
            for i in values:
                [header,result]=voltage(model,structure,li_concentration,functional_list,i,j,configuration)
                plot[p][q]=result[0][1]
                q = q + 1
            q = 0
            p = p + 1

        for j in configurations:
            i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
            plt.plot(i,plot[j],label='Conf_'+str(j),linestyle='--',marker='o')
    else:
        plot = [0] * len(values)
        q=0
        for i in values:
            [header,result]=voltage(model,structure,li_concentration,functional_list,i,conf,configuration)
            plot[q]=result[0][1]
            q = q + 1   

        i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
        plt.plot(i,plot,label='Conf_'+str(conf),linestyle='--',marker='o')


    ax.legend()    
    my_xticks = (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)
    i = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    plt.xticks(i, my_xticks)
    plt.xlabel('U-value (eV)')
    plt.ylabel('Voltage (V)')
    plt.title('Voltage for different configurations and U-values')
    plt.show()

In [94]:
def conf_checker(model,li_concentration,structure):
    if (model == 'Li2MnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:],structure)
    elif (model == 'Li2IrO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,model[0:2]+li_concentration+model[3:])
    elif (model == 'Li2MnSnO3'):
        dir = os.path.join('/scratch/antwerpen/204/vsc20412',model,'Li'+li_concentration)
        
    check = []
    for x in range(0,15):
        directory=os.path.join(dir,"conf_"+str(x))
        if (os.path.exists(directory) == True):
            check.append(x)
    if check == []:
        check = [0]
    else:
        check = check
        
    return check

In [95]:
import ipywidgets as widgets
from ipywidgets import interact
from energy_check import energy_check
from conf_checker import conf_checker
from out_checker import out_checker
from energy_check import energ
from baseline_energy import
from magnetic_check import
from baseline_energy import
from visualization_energy_difference import
from visualization_lattice import
from lattice_check import
from visualization import
from show_structure import
from magnetic_plot import
from voltage import
from voltage_visualization import

output = widgets.Output()
plot_output = widgets.Output()
plot1_output = widgets.Output()

def common_filtering(model, structure, li_concentration, functional_list,U_value,what,conf,visualize,figure,configuration,energy_baseline):
  
    output.clear_output()
    plot_output.clear_output()
    plot1_output.clear_output()
    
    if (model == 'None'):
        common_filter = 'Please chose a material.'
    elif (len(conf_checker(model,li_concentration,structure)) <= conf):
        common_filter = "This material in combination with this structure does not have this configuration. Try a lower number."
    elif (model != 'Li2MnO3') & (structure == 'O1'):
        common_filter = 'The O1-stacking is only an option for Li2MnO3.'
    elif (what == 'Final Energy') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,energy] = energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        common_filter = tabulate(energy, headers=header, tablefmt="latex",floatfmt=(".1f",".2f"))
    elif (what == 'out - required accuracy') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        common_filter = out_checker(model,li_concentration,structure,functional_list,U_value,conf,configuration)
    elif (what == 'Magnetic moments') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,results] = magnetic_check(model,structure,li_concentration,functional_list,U_value,conf)
        common_filter = tabulate(results, headers=header, tablefmt="latex", floatfmt=".2f")
    elif (what == 'lattice parameters') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,lattice_parameters_length] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)
        common_filter = tabulate(lattice_parameters_length, headers=header, tablefmt="latex", floatfmt=(".1f",".5f"))
    elif (what == 'lattice angles') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,lattice_parameters_angles] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)
        common_filter = tabulate(lattice_parameters_angles, headers=header, tablefmt="latex", floatfmt=(".1f",".5f"))
    elif (what == 'voltage') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,result] = voltage(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        common_filter = tabulate(result, headers=header, tablefmt="latex", floatfmt=(".1f",".2f"))
    
    with output:
        if (what == 'out - required accuracy'):
            display(common_filter)
        else:
            print(common_filter)
            
    with plot_output:
        if (len(conf_checker(model,li_concentration,structure)) <= conf):
            print("This material in combination with this structure does not have this configuration. Try a lower number.")
        elif (U_value == (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)) & (functional_list == 'pbeU'):
            print('Cannot visualize multiple files. Change the U-value setting.')
        elif (model == 'None'):
            print('Please chose a material.')
        elif (visualize != True):
            print('If you want to viualize a structure check the Visualize Structure checkbox.')
        elif (visualize == True):
            visualization(model,li_concentration,structure,functional_list,U_value,conf)
              
    with plot1_output: 
        if (model == 'None'):
            print('Please chose a material.')
        elif (len(conf_checker(model,li_concentration,structure)) <= conf):
            print("This material in combination with this structure does not have this configuration. Try a lower number.")
        elif (figure != True):
            print('If you want to viualize a structure check the Figures checkbox.')
        elif (U_value != (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)):
            print('The figures are only interesting for all U-values.')
        elif (what == 'voltage') & (figure == True) & (U_value != (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)):
            print('The figures are only interesting for all U-values.')
        elif (what == 'lattice parameters') & (figure == True):
            visualization_lattice(model,li_concentration,structure,functional_list,U_value,conf)
        elif (what == 'lattice angles') & (figure == True):
            visualization_lattice(model,li_concentration,structure,functional_list,U_value,conf)
        elif (what == 'Final Energy') & (figure == True):
            visualization_energy_difference(model,li_concentration,structure,functional_list,U_value,conf,configuration,energy_baseline)
        elif (what == 'Magnetic moments') & (figure == True):
            magnetic_plot(model,li_concentration,structure,functional_list,U_value,conf)
        elif (what == 'voltage') & (figure == True) & (U_value == (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)):
            voltage_visualization(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        

In [96]:
u_values = ((0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6),0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6),

interact(common_filtering, 

         
# All the defenitions of the dropdown menu's
model = widgets.Dropdown(
    options=["None","Li2MnO3", "Li2IrO3","Li2MnSnO3"],
    description='Material:',
    disabled=False,
),
         
structure = widgets.Dropdown(
    options=['prim','conv','O1'],
    description='Cell: (only for Li2MnO3)',
    disabled=False,
),

li_concentration = widgets.Dropdown(
    options=["0.5","2"],
    description='Li conc:',
    disabled=False,
),

functional_list = widgets.Dropdown(
    options=["pbeU", "scan" , "hse",("pbeU", "scan"),("pbeU", "scan" , "hse")],
    description='Functional:',
    disabled=False,
),


U_value = widgets.Dropdown(
    options=u_values, 
    description='U-value: (only for pbeU)',
    disabled=False,
),

what = widgets.Dropdown(
    options= ["Final Energy", "Magnetic moments","lattice parameters","lattice angles","voltage","out - required accuracy"],
    description='What?:',
    disabled=False,
),

conf = widgets.Dropdown(
    #options= conf_checker(models,li_concentrations,structures),
    options = (0,1,2,3,4,5,6,7,8,9,10),
    description='Conf:',
    disabled=False,
),

energy_baseline = widgets.Dropdown(
    options= ["Discharged", "Absolute","Minimum"],
    description='Baseline for Final Energy:',
    disabled=False,
),

configuration = widgets.Checkbox(
           description='Conf: All.',),

visualize = widgets.Checkbox(
           description='Unlock: Visualize structure.',),

figure = widgets.Checkbox(
           description='Unlock: Figures.',)
)
output

interactive(children=(Dropdown(description='Material:', options=('None', 'Li2MnO3', 'Li2IrO3', 'Li2MnSnO3'), v…

Output()

In [99]:
def running(test):
    if test == 'On':
        print('succes')
        result=conf_checker('Li2MnO3','0.5','prim')
        print(result)
    else:
        print('fail')

interact(running, test = widgets.Dropdown(
    options=["On","Off"],
    description='Material:',
    disabled=False,
))

interactive(children=(Dropdown(description='Material:', options=('On', 'Off'), value='On'), Output()), _dom_cl…

<function __main__.running(test)>